In [1]:
# Define F2 algebra
import Base.+
import Base.*

# FALSE ~ 0, TRUE ~ 1
*(A::Bool, B::Bool) = A && B # equivlant to and
# 1 * 1 = 1, 0 * 1 = 0 * 0 = 1 * 0 = 0
+(A::Bool, B::Bool) = xor(A, B) # equivlant to exclusive or 
# 1 + 1 = 0 + 0 = 0,  0 + 1 = 1 + 0 = 1

+ (generic function with 180 methods)

(Bool, Bool) in module Base at bool.jl:91 overwritten in module Main at In[1]:6.


In [2]:
b2i(matrix) = [x? 1 : 0 for x in matrix]
i2b(matrix) = matrix.> 0

i2b (generic function with 1 method)

In [3]:
function swap_row!(H, i, j)
    H[i, :], H[j, :] = H[j, :], H[i, :]
end

function gaussian_elimination(H)
    n_minus_k, n = size(H)
    k = n - n_minus_k
    # k: length of an input, n: length of a code, 
    # n_minus_k: length of redunance (num of rows of decoding matrix)
    if rank(H[:, 1 : n_minus_k]) == n_minus_k
        H1 = H .> 0
        
        # ensure the main variable (that variable in the matrix angle)
        # has value of 1 (or say, value of true)
        
        for col in 1: n_minus_k
            true_row = -1
            the_row = col
            for r in the_row : n_minus_k
                if H1[r, col] == true
                    true_row = r
                    break
                end
            end
            swap_row!(H1, true_row, the_row)
            
            @show b2i(H1)
            for r in the_row + 1: n_minus_k
                if H1[r, col]
                   H1[r, :] += H1[the_row, :]
                end
            end
            @show b2i(H1)
        end
        
        #reverse elimiation
        for row in reverse(1:n_minus_k)
            col = row
            for r in reverse(1: row - 1)
                if H1[r, col]
                    H1[r, :] += H1[row, :]
                end
            end
        end
    else
        throw("unimplemented")
        #TO DO, implement for a none-full rank matrix
    end
    #@show b2i(H1)
    return H1
end


function get_encoder(H)
    
    n_minus_k, n = size(H)
    k = n - n_minus_k
    
    # k: length of an input, n: length of a code, 
    # n_minus_k: length of redunance (num of rows of decoding matrix)
    
    H1 = gaussian_elimination(H)
    P = H1[:, n_minus_k + 1: n]
    G = vcat(P, i2b(eye(k, k)))
    return H1, G
end

get_encoder (generic function with 1 method)

In [4]:
H = 
[1 1 1 1 0 0
 0 0 1 1 0 1
 1 0 0 1 1 0]
H = i2b(H)
H1, G = get_encoder(H)

@show b2i(H1)
@show b2i(G)

b2i(H1) = [1 1 1 1 0 0; 0 0 1 1 0 1; 1 0 0 1 1 0]
b2i(H1) = [1 1 1 1 0 0; 0 0 1 1 0 1; 0 1 1 0 1 0]
b2i(H1) = [1 1 1 1 0 0; 0 1 1 0 1 0; 0 0 1 1 0 1]
b2i(H1) = [1 1 1 1 0 0; 0 1 1 0 1 0; 0 0 1 1 0 1]
b2i(H1) = [1 1 1 1 0 0; 0 1 1 0 1 0; 0 0 1 1 0 1]
b2i(H1) = [1 1 1 1 0 0; 0 1 1 0 1 0; 0 0 1 1 0 1]
b2i(H1) = [1 0 0 1 1 0; 0 1 0 1 1 1; 0 0 1 1 0 1]
b2i(G) = [1 1 0; 1 1 1; 1 0 1; 1 0 0; 0 1 0; 0 0 1]


6×3 Array{Int64,2}:
 1  1  0
 1  1  1
 1  0  1
 1  0  0
 0  1  0
 0  0  1

In [1]:
test = 
[
    true
    false
    true
]
@show G * test
H1 * G

LoadError: [91mUndefVarError: G not defined[39m

In [6]:
# Define [P(x = false | y), P(x = true | y)]
p_y_g_x(y, q) = [q^(y + false) * (1 - q)^(y + false + true), q^(y + true) * (1 - q)^(y + true + true)]

p_y_g_x (generic function with 1 method)

In [7]:
# All combination of states for neighboring variable in one factor (or say, equation)
allcombination(k) = [[ bit == '1' ? true : false for bit in bin(n,k) ] for n in 0:2^k-1]

allcombination (generic function with 1 method)

In [8]:
# Helper function
function is_sum_zero(b_arr, this_bool)
    res = this_bool
    for b in b_arr
        res = res + b
    end
    return res == false
end

is_sum_zero (generic function with 1 method)

In [9]:
# Normalise each message to a probablity distribution
# This is essential to avoid underflow
function normalise_message!(message)
    m, n, n_p = size(message)
    for i in 1:m
        for j in 1:n
            message[i, j, :] = message[i, j, :] / sum(message[i, j, :])
        end
    end
    return message
end

normalise_message! (generic function with 1 method)

In [10]:
# Passing message (vector or say distribution) to variable
# Parameters：
# m: number of factors (#rows of H matrix)
# n: number of variables (#cols of H matrix)
# factors: a list of variables for different factor
# messages: messages passed from varible to factor
function factor2variable(m, n, factors, messages)
    new_mes = zeros(m, n, 2)
    
    for factor_ind in eachindex(factors)
        for variable in factors[factor_ind]
            # For each message that from factor_{factor_ind} -> variable
            
            other_variables = setdiff(factors[factor_ind], [variable])
            # other neighboring varibles in the factor
            
            all_combination = allcombination(size(other_variables)[1])
            for variable_state in [false, true]
                for combination in all_combination
                    # ensure the sum of states is 0 (i.e. false)
                    if is_sum_zero(combination, variable_state)
                       # calculate message from factor_{factor_ind} -> variable
                       # for the variable in either state 0 or state 1
                       temp = 1
                       for other_var_ind in eachindex(other_variables)
                          temp = temp * messages[factor_ind, 
                                other_variables[other_var_ind], 
                                combination[other_var_ind]?2:1]
                       end
                       if size(other_variables)[1] > 1
                            new_mes[factor_ind, variable, variable_state?2:1] += temp
                       end
                    end  
                end
            end
        end         
    end
    return normalise_message!(new_mes)
end


# Passing message (vector or say distribution) to factor
# Parameters：
# m: number of factors (#rows of H matrix)
# n: number of variables (#cols of H matrix)
# factors: a list of variables for different factor
# record_probability: P(y_n | x_n), constant
# messages: messages passed from factor to varible
function variable2factor(m, n, factors, messages, record_probability)
    # messages an m * n * 2 matrix
    marginal = zeros(size(messages)[2], size(messages)[3])
    new_mes = zeros(size(messages))
    messages = normalise_message!(messages)
    for factor_ind in eachindex(factors)
        for variable in factors[factor_ind]
            temp = [1, 1]
            for ano_factor_ind in eachindex(factors)
                if variable in factors[ano_factor_ind] && ano_factor_ind != factor_ind
                    temp = temp .* messages[ano_factor_ind, variable, :]
                end
            end
            new_mes[factor_ind, variable, :] = record_probability[variable, :] .* temp
            marginal[variable, :] += new_mes[factor_ind, variable, :] .* messages[factor_ind, variable, :]
        end
    end
    return normalise_message!(new_mes), marginal
    
end

variable2factor (generic function with 1 method)

In [11]:
function decode(H, y, p = 0.1, n_iter = 20)
    
    n_minus_k, n = size(H)
    k = n - n_minus_k
    
    H = H.>0
    y = y.>0    
    
    factors = [Int[] for i = 1: n_minus_k]
    
    # set factor list
    for i in 1: n_minus_k
        for j in 1:n
            if H[i, j]
                push!(factors[i], j)
            end
        end
    end
    
    # set P(y|x)
    record_probability = zeros(n, 2)
    for j in 1:n
        record_probability[j, :] = p_y_g_x(y[j], p)
    end
    
    # Step 1 initalise message
    messages = zeros(n_minus_k, n, 2)
    for i in 1:n_minus_k
        for j in 1:n
            messages[i, j, :] = p_y_g_x(y[j], p)
        end
    end
    
    marginals = 0
    
    for ith in 1:n_iter
        #println(ith)
        # Step 2 factor -> variable message passing
        messages = factor2variable(n_minus_k, n, factors, messages)
        # Step 3 variable -> factor message passing
        messages, marginals = variable2factor(n_minus_k, n, factors, messages, record_probability)
        posterior = [(marginals[i, 1] > marginals[i, 2]? false: true) for i in 1: n]
        if all(!, H * posterior) 
           #println("successful")
           return 0, posterior, record_probability, ith
        end
    end
    
    last_posterior = [(marginals[i, 1] > marginals[i, 2]? 0: 1) for i in 1: n]
    return -1,last_posterior, record_probability, n_iter
    
end

decode (generic function with 3 methods)

In [12]:
H = readdlm("H1.txt")
y = readdlm("y1.txt")
result_code,  posterior, record_probability = @time decode(H, y)
showall(b2i(posterior))

 21.396332 seconds (180.92 M allocations: 7.403 GiB, 6.63% gc time)
[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0

In [13]:
decoded = [posterior[start : start + 7] for start in range(1, 8, 31)]
res = Vector{Char}(0)
for array in decoded
    temp = Vector{Char}(0)
    for i in 1: length(array)
        push!(temp, array[i]>0?'1':'0')
    end
    push!(res, Char(parse(BigInt,String(temp),2)))
end

In [14]:
String(res)

"Happy Holidays! Dmitry&David :)"

In [15]:
function binary_channel_passing(p, x)
    errorbits = [rand() < p for i in collect(1: length(x))]
    y = []
    for bit in collect(1:length(x))
        append!(y, errorbits[bit]? 1 - x[bit] : x[bit])
    end
    return y
end

binary_channel_passing (generic function with 1 method)

In [16]:
function str2binstr(str)
    binstr = ""
    for c in str
        binstr = string(binstr, bin(c, 8))
    end
    return string(binstr, "1111") # padding to 252
end

str2binarr(str) = [i == '1'?1:0 for i in str2binstr(str)]

str2binarr (generic function with 1 method)

In [17]:
G = readdlm("sys_G.txt")

1000×252 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0

In [18]:
function encoding_str_using_G(str) 
   return b2i(i2b(G) * i2b(str2binarr(str))) # x = G * t
end

encoding_str_using_G (generic function with 1 method)

In [19]:
function encoding_using_G(arr)
   return b2i(i2b(G) * i2b(arr)) # x = G * t
end

encoding_using_G (generic function with 1 method)

In [20]:
function produce_fake_xs(num_data)
    res = Array{Array{Int}}(0)
    for j in collect(1: num_data)
        temp = Array{Int}(0)
        for i in collect(1: 252)
            push!(temp, (rand() < 0.5)?1:0)
        end
        push!(res, temp)
    end
    return res
end
produce_fake_xs(1000)

res = Array{Int64,N} where N[]


1000-element Array{Array{Int64,N} where N,1}:
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 1  …  1, 1, 0, 1, 1, 0, 0, 0, 1, 0]
 [0, 0, 0, 0, 1, 0, 0, 1, 1, 1  …  0, 0, 1, 1, 0, 1, 0, 1, 0, 1]
 [1, 0, 1, 0, 1, 1, 0, 0, 0, 0  …  1, 1, 0, 1, 1, 1, 0, 1, 1, 1]
 [1, 1, 0, 0, 0, 1, 1, 1, 0, 0  …  1, 1, 0, 0, 1, 1, 1, 0, 1, 0]
 [0, 1, 0, 0, 1, 1, 0, 0, 1, 0  …  0, 1, 1, 0, 1, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 1, 0, 0, 1, 1  …  1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 1, 1, 0, 0, 1, 1  …  1, 1, 1, 1, 1, 1, 0, 1, 1, 0]
 [0, 1, 1, 1, 1, 1, 0, 0, 0, 0  …  0, 1, 1, 1, 1, 0, 0, 0, 0, 1]
 [0, 1, 1, 0, 1, 1, 1, 1, 0, 0  …  0, 0, 1, 1, 0, 0, 1, 1, 0, 1]
 [0, 1, 0, 1, 1, 0, 0, 0, 1, 1  …  1, 0, 0, 1, 0, 0, 0, 0, 0, 1]
 [1, 0, 1, 1, 0, 1, 1, 0, 0, 1  …  1, 1, 0, 1, 1, 0, 1, 1, 1, 0]
 [0, 0, 0, 1, 1, 0, 0, 0, 1, 0  …  1, 1, 1, 1, 0, 0, 0, 0, 1, 1]
 [0, 0, 0, 1, 0, 1, 0, 0, 0, 0  …  1, 1, 1, 0, 1, 0, 1, 0, 1, 1]
 ⋮                                                              
 [1, 0, 1, 1, 0, 0, 0, 1, 1, 0  …  1, 1, 1, 

In [21]:
function empirical_study(n_data = 100)
    ts = produce_fake_xs(n_data)
    res = [];
    ps = []
    for p in 0.05:0.02:0.16
        success_acc = 0
        push!(ps, p)
        println(p)
        for t in ts
            x = encoding_using_G(t)
            y = binary_channel_passing(p, x)
            @time result_code,  posterior, record_probability, n_iterate = decode(H, y)
            if result_code >= 0 
                success_acc += 1
            end
        end
        push!(res, success_acc / n_data)
    end
    return ps, res
end

empirical_study (generic function with 2 methods)

In [22]:
ps, res = empirical_study()

res = Array{Int64,N} where N[]
0.05
  7.768479 seconds (71.51 M allocations: 2.874 GiB, 8.15% gc time)
  9.614758 seconds (93.47 M allocations: 3.776 GiB, 7.68% gc time)
  7.370150 seconds (71.47 M allocations: 2.872 GiB, 8.39% gc time)
  9.394661 seconds (71.47 M allocations: 2.872 GiB, 7.11% gc time)
  9.968972 seconds (71.47 M allocations: 2.872 GiB, 5.74% gc time)
  9.388809 seconds (71.47 M allocations: 2.872 GiB, 7.70% gc time)
  8.007151 seconds (71.47 M allocations: 2.872 GiB, 8.07% gc time)
 12.927700 seconds (93.47 M allocations: 3.776 GiB, 6.97% gc time)
 10.249214 seconds (71.47 M allocations: 2.872 GiB, 7.72% gc time)
 14.697238 seconds (93.47 M allocations: 3.776 GiB, 6.48% gc time)
  5.821273 seconds (49.47 M allocations: 1.968 GiB, 7.24% gc time)
 14.579955 seconds (93.47 M allocations: 3.776 GiB, 6.53% gc time)
  9.456073 seconds (71.47 M allocations: 2.872 GiB, 6.09% gc time)
 11.729367 seconds (71.47 M allocations: 2.872 GiB, 6.77% gc time)
 15.602370 seconds (93.47 

 10.703619 seconds (93.47 M allocations: 3.776 GiB, 8.33% gc time)
 12.486048 seconds (93.47 M allocations: 3.776 GiB, 7.16% gc time)
 12.545251 seconds (115.46 M allocations: 4.680 GiB, 8.22% gc time)
  7.599640 seconds (71.47 M allocations: 2.872 GiB, 8.49% gc time)
 10.012754 seconds (93.47 M allocations: 3.776 GiB, 7.79% gc time)
 12.249401 seconds (115.46 M allocations: 4.680 GiB, 8.24% gc time)
 10.220897 seconds (93.47 M allocations: 3.776 GiB, 8.11% gc time)
  9.970664 seconds (93.47 M allocations: 3.776 GiB, 7.68% gc time)
  9.839127 seconds (93.47 M allocations: 3.776 GiB, 7.81% gc time)
  9.988673 seconds (93.47 M allocations: 3.776 GiB, 8.82% gc time)
  9.830231 seconds (93.47 M allocations: 3.776 GiB, 7.75% gc time)
 12.178081 seconds (115.46 M allocations: 4.680 GiB, 8.26% gc time)
  9.820260 seconds (93.47 M allocations: 3.776 GiB, 7.78% gc time)
 10.094901 seconds (93.47 M allocations: 3.776 GiB, 7.92% gc time)
 10.413331 seconds (93.47 M allocations: 3.776 GiB, 7.46% g

  6.617030 seconds (115.46 M allocations: 4.680 GiB, 8.82% gc time)
  9.730018 seconds (159.46 M allocations: 6.489 GiB, 9.49% gc time)
  6.828848 seconds (115.46 M allocations: 4.680 GiB, 9.75% gc time)
  7.646273 seconds (137.46 M allocations: 5.585 GiB, 9.43% gc time)
  5.115866 seconds (93.47 M allocations: 3.776 GiB, 9.65% gc time)
  6.459111 seconds (115.46 M allocations: 4.680 GiB, 9.82% gc time)
  5.121303 seconds (93.47 M allocations: 3.776 GiB, 9.41% gc time)
  6.314697 seconds (115.46 M allocations: 4.680 GiB, 10.05% gc time)
  6.460876 seconds (115.46 M allocations: 4.680 GiB, 10.10% gc time)
  7.842085 seconds (115.46 M allocations: 4.680 GiB, 9.45% gc time)
  7.932273 seconds (137.46 M allocations: 5.585 GiB, 9.52% gc time)
  6.310774 seconds (115.46 M allocations: 4.680 GiB, 10.05% gc time)
  7.539933 seconds (137.46 M allocations: 5.585 GiB, 9.59% gc time)
  9.822406 seconds (181.45 M allocations: 7.393 GiB, 9.61% gc time)
  6.327221 seconds (115.46 M allocations: 4.680

  7.555000 seconds (137.46 M allocations: 5.585 GiB, 10.70% gc time)
 12.322383 seconds (225.45 M allocations: 9.201 GiB, 9.61% gc time)
  8.674185 seconds (159.46 M allocations: 6.489 GiB, 9.98% gc time)
 10.994685 seconds (203.45 M allocations: 8.297 GiB, 9.35% gc time)
  7.531135 seconds (137.46 M allocations: 5.585 GiB, 10.45% gc time)
  8.614724 seconds (159.46 M allocations: 6.489 GiB, 9.22% gc time)
 11.055182 seconds (203.45 M allocations: 8.297 GiB, 9.96% gc time)
  9.812113 seconds (181.45 M allocations: 7.393 GiB, 9.63% gc time)
  7.725868 seconds (137.46 M allocations: 5.585 GiB, 10.27% gc time)
 10.025967 seconds (181.45 M allocations: 7.393 GiB, 9.49% gc time)
  9.883185 seconds (181.45 M allocations: 7.393 GiB, 9.58% gc time)
 11.047882 seconds (203.45 M allocations: 8.297 GiB, 9.94% gc time)
  7.472484 seconds (137.46 M allocations: 5.585 GiB, 9.56% gc time)
  9.816984 seconds (181.45 M allocations: 7.393 GiB, 9.63% gc time)
 12.283976 seconds (225.45 M allocations: 9.2

 17.448117 seconds (247.44 M allocations: 10.105 GiB, 8.62% gc time)
 12.733255 seconds (181.45 M allocations: 7.393 GiB, 9.31% gc time)
 15.679181 seconds (247.44 M allocations: 10.105 GiB, 8.90% gc time)
 14.082010 seconds (225.45 M allocations: 9.201 GiB, 9.88% gc time)
 25.998738 seconds (445.42 M allocations: 18.242 GiB, 9.49% gc time)
 12.812334 seconds (225.45 M allocations: 9.201 GiB, 9.57% gc time)
 12.536710 seconds (225.45 M allocations: 9.201 GiB, 9.55% gc time)
 13.065054 seconds (225.45 M allocations: 9.201 GiB, 10.13% gc time)
  8.101656 seconds (137.46 M allocations: 5.585 GiB, 9.36% gc time)
 17.924351 seconds (291.44 M allocations: 11.913 GiB, 10.00% gc time)
 10.432114 seconds (181.45 M allocations: 7.393 GiB, 9.38% gc time)
 19.079289 seconds (335.43 M allocations: 13.721 GiB, 9.86% gc time)
 18.389011 seconds (313.43 M allocations: 12.817 GiB, 9.46% gc time)
 15.825672 seconds (269.44 M allocations: 11.009 GiB, 9.51% gc time)
 15.535484 seconds (269.44 M allocation

(Any[0.05, 0.07, 0.09, 0.11, 0.13, 0.15], Any[1.0, 1.0, 1.0, 1.0, 0.94, 0.51])

In [29]:
ts = produce_fake_xs(100)
res2 = [];
ps2 = []
for p in 0.15:0.02:0.22
    success_acc = 0
    push!(ps2, p)
    println(p)
    for t in ts
        x = encoding_using_G(t)
        y = binary_channel_passing(p, x)
        @time result_code,  posterior, record_probability, n_iterate = decode(H, y)
        if result_code >= 0 
            success_acc += 1
        end
    end
    push!(res2, success_acc / 100)
end

res = Array{Int64,N} where N[]
0.15
 26.601092 seconds (247.44 M allocations: 10.105 GiB, 8.70% gc time)
 47.694918 seconds (445.42 M allocations: 18.242 GiB, 8.46% gc time)
 47.500894 seconds (445.42 M allocations: 18.242 GiB, 8.58% gc time)
 29.224998 seconds (269.44 M allocations: 11.009 GiB, 8.51% gc time)
 47.746423 seconds (445.42 M allocations: 18.242 GiB, 8.79% gc time)
 24.143387 seconds (225.45 M allocations: 9.201 GiB, 8.49% gc time)
 38.183939 seconds (357.43 M allocations: 14.625 GiB, 8.51% gc time)
 35.996473 seconds (335.43 M allocations: 13.721 GiB, 8.69% gc time)
 47.313204 seconds (445.42 M allocations: 18.242 GiB, 8.56% gc time)
 47.201481 seconds (445.42 M allocations: 18.242 GiB, 8.50% gc time)
 47.119618 seconds (445.42 M allocations: 18.242 GiB, 8.84% gc time)
 30.701469 seconds (291.44 M allocations: 11.913 GiB, 8.35% gc time)
 33.711774 seconds (313.43 M allocations: 12.817 GiB, 8.96% gc time)
 33.641578 seconds (313.43 M allocations: 12.817 GiB, 8.55% gc time)

 26.209944 seconds (445.42 M allocations: 18.242 GiB, 10.21% gc time)
 26.179908 seconds (445.42 M allocations: 18.242 GiB, 10.10% gc time)
 26.284997 seconds (445.42 M allocations: 18.242 GiB, 10.54% gc time)
 27.601105 seconds (445.42 M allocations: 18.242 GiB, 9.76% gc time)
 26.272891 seconds (445.42 M allocations: 18.242 GiB, 10.16% gc time)
 25.950284 seconds (445.42 M allocations: 18.242 GiB, 10.28% gc time)
 26.061951 seconds (445.41 M allocations: 18.241 GiB, 10.44% gc time)
 25.709963 seconds (445.42 M allocations: 18.242 GiB, 10.30% gc time)
 25.897554 seconds (445.42 M allocations: 18.242 GiB, 10.23% gc time)
 26.454775 seconds (445.42 M allocations: 18.242 GiB, 10.17% gc time)
 25.903692 seconds (445.42 M allocations: 18.242 GiB, 10.53% gc time)
 25.826100 seconds (445.42 M allocations: 18.242 GiB, 10.22% gc time)
 25.929341 seconds (445.42 M allocations: 18.242 GiB, 10.25% gc time)
 26.301295 seconds (445.42 M allocations: 18.242 GiB, 10.22% gc time)
 27.152620 seconds (4

 34.982770 seconds (445.42 M allocations: 18.242 GiB, 9.53% gc time)
 29.671424 seconds (445.42 M allocations: 18.242 GiB, 9.67% gc time)
 30.492009 seconds (445.42 M allocations: 18.242 GiB, 9.67% gc time)
 29.912374 seconds (445.42 M allocations: 18.242 GiB, 9.62% gc time)
 29.515892 seconds (445.42 M allocations: 18.242 GiB, 10.45% gc time)
 31.345313 seconds (445.42 M allocations: 18.242 GiB, 9.35% gc time)
 33.904842 seconds (445.42 M allocations: 18.242 GiB, 9.24% gc time)
 28.387308 seconds (445.42 M allocations: 18.242 GiB, 9.94% gc time)
 32.143687 seconds (445.42 M allocations: 18.242 GiB, 9.60% gc time)
 30.569919 seconds (445.42 M allocations: 18.242 GiB, 9.34% gc time)
 31.056302 seconds (445.42 M allocations: 18.242 GiB, 9.35% gc time)
 29.217148 seconds (445.42 M allocations: 18.242 GiB, 10.02% gc time)
 29.165230 seconds (445.42 M allocations: 18.242 GiB, 10.01% gc time)
 28.369680 seconds (445.42 M allocations: 18.242 GiB, 9.78% gc time)
 28.187947 seconds (445.42 M al

 32.562651 seconds (445.42 M allocations: 18.242 GiB, 9.39% gc time)
 29.398038 seconds (445.42 M allocations: 18.242 GiB, 10.31% gc time)
 26.790824 seconds (445.42 M allocations: 18.242 GiB, 10.25% gc time)
 26.797394 seconds (445.42 M allocations: 18.242 GiB, 10.16% gc time)
 26.779367 seconds (445.42 M allocations: 18.242 GiB, 10.15% gc time)
 26.826634 seconds (445.42 M allocations: 18.242 GiB, 10.48% gc time)
 29.168076 seconds (445.42 M allocations: 18.242 GiB, 9.65% gc time)
 28.655526 seconds (445.42 M allocations: 18.242 GiB, 10.08% gc time)
 25.525711 seconds (445.42 M allocations: 18.242 GiB, 10.28% gc time)
 25.400488 seconds (445.42 M allocations: 18.242 GiB, 10.78% gc time)
 25.293442 seconds (445.42 M allocations: 18.242 GiB, 10.31% gc time)
 25.352942 seconds (445.42 M allocations: 18.242 GiB, 10.32% gc time)
 26.318281 seconds (445.42 M allocations: 18.242 GiB, 10.07% gc time)
 32.061800 seconds (445.42 M allocations: 18.242 GiB, 9.58% gc time)
 27.877955 seconds (445

In [32]:
@show res2
@show ps2

res2 = Any[0.56, 0.12, 0.0, 0.0]
ps2 = Any[0.15, 0.17, 0.19, 0.21]


4-element Array{Any,1}:
 0.15
 0.17
 0.19
 0.21

In [ ]:
test_phase_1 = "Hi! Welcome to the Julia World."
x = encoding_str_using_G(test_phase_1)
@show x
y = binary_channel_passing(0.145, x)
@show y - x

In [ ]:
result_code,  posterior, record_probability, n_iterate = decode(H, y)